In [4]:
import pickle
with open("documenttop1_text_list.txt", "rb") as f:
    document_top1 = pickle.load(f)

In [3]:
from nltk.corpus import opinion_lexicon
neg = list(opinion_lexicon.negative())
pos = list(opinion_lexicon.positive())

In [5]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

In [6]:
# convert documents list to string
doc = ''.join(document_top1)
tokens = tokenizer.tokenize(doc)

In [7]:
from stop_words import get_stop_words

# create English stop words list
en_stop = get_stop_words('en')

In [8]:
# remove stop words from tokens
stopped_tokens = [i for i in tokens if not i in en_stop]

In [9]:
from nltk.stem.porter import PorterStemmer

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
# stem token
texts = [p_stemmer.stem(i) for i in stopped_tokens]

In [10]:
from nltk.corpus import opinion_lexicon
neg = list(opinion_lexicon.negative())
pos = list(opinion_lexicon.positive())

In [13]:
# def of adding codeword
def codeword(words):
    for i in range(len(words)):
        if words[i] in pos:
            words.insert(i+1, "GOODREVIEW")
        if words[i] in neg:
            words.insert(i+1, "NEGREVIEW")

In [14]:
codeword(texts)

In [15]:
texts[1:50]

['chic',
 'GOODREVIEW',
 'although',
 'menu',
 'item',
 'doesnt',
 'scream',
 'NEGREVIEW',
 'french',
 'cuisin',
 'most',
 'food',
 'look',
 'like',
 'GOODREVIEW',
 'can',
 'get',
 'american',
 'place',
 'the',
 'food',
 'awesom',
 'though',
 'one',
 'place',
 'I',
 'actual',
 'come',
 'satisfi',
 'I',
 'order',
 'smoke',
 'NEGREVIEW',
 'salmon',
 'platter',
 'I',
 'enjoy',
 'GOODREVIEW',
 'I',
 'enjoy',
 'GOODREVIEW',
 'simpl',
 'food',
 'My',
 'plate',
 'shred',
 'salmon',
 'pile',
 'caper']

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
no_features = 1000
# # Use tf-idf features for NMF.
# print("Extracting tf-idf features for NMF...")
# tfidf_vectorizer = TfidfVectorizer(max_features=no_features,
#                                    stop_words='english')
# tfidf = tfidf_vectorizer.fit_transform(texts)
# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_features=no_features, stop_words='english')
tf = tf_vectorizer.fit_transform(texts)
tf_feature_names = tf_vectorizer.get_feature_names()
type(tfidf)

Extracting tf-idf features for NMF...


scipy.sparse.csr.csr_matrix

In [78]:
# topic words = 10
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic %d:" % (topic_idx))
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 10

In [83]:
from sklearn.decomposition import LatentDirichletAllocation
no_topics = 6

# # Run NMF
# nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

# # Run LDA
lda_codeword = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)


In [84]:
display_topics(lda_codeword, tf_feature_names, no_top_words)

Topic 0:
place vega just bellagio love tri bread outsid frite amaz
Topic 1:
food order restaur come dinner gabi meal enjoy thi lunch
Topic 2:
servic time patio came breakfast ve perfect want onion salad
Topic 3:
negreview great steak like wait tabl ami strip mon recommend
Topic 4:
goodreview french fri right menu ask look better locat everyth
Topic 5:
good seat view realli delici nice fountain got reserv best


In [85]:
def get_topics(model, feature_names, no_top_words):
    topic_lists = []
    for topic_idx, topic in enumerate(model.components_):
        topic_list = []
        for i in topic.argsort()[:-no_top_words - 1:-1]:
            topic_list += [feature_names[i]]
        topic_lists.append(topic_list)
    return topic_lists
d = get_topics(lda_codeword, tf_feature_names, no_top_words)



In [88]:
import texttable as tt
tab = tt.Texttable()
headings = ['food','order','service','bad price','good environment','place']
tab.header(headings)
d0 = d[0]
d1 = d[1]
d2 = d[2]
d3 = d[3]
d4 = d[4]
d5 = d[5]
# d6 = d[6]
# d7 = d[7]
# d8 = d[8]
# d9 = d[9]


for row in zip(d0,d1,d2,d3,d4,d5):
    tab.add_row(row)

s = tab.draw()
print (s)

+----------+---------+-----------+-----------+------------------+----------+
|   food   |  order  |  service  | bad price | good environment |  place   |
+==========+=========+===========+===========+==================+==========+
| place    | food    | servic    | negreview | goodreview       | good     |
+----------+---------+-----------+-----------+------------------+----------+
| vega     | order   | time      | great     | french           | seat     |
+----------+---------+-----------+-----------+------------------+----------+
| just     | restaur | patio     | steak     | fri              | view     |
+----------+---------+-----------+-----------+------------------+----------+
| bellagio | come    | came      | like      | right            | realli   |
+----------+---------+-----------+-----------+------------------+----------+
| love     | dinner  | breakfast | wait      | menu             | delici   |
+----------+---------+-----------+-----------+------------------+----------+

In [89]:
# topic words = 20
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic %d:" % (topic_idx))
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 20
no_topics = 6

# # Run NMF
# nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

# # Run LDA
lda_codeword = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)
display_topics(lda_codeword, tf_feature_names, no_top_words)

Topic 0:
place vega just bellagio love tri bread outsid frite amaz peopl make soup experi tast butter friendli drink awesom disappoint
Topic 1:
food order restaur come dinner gabi meal enjoy thi lunch alway went dish star night don pari warm bit minut
Topic 2:
servic time patio came breakfast ve perfect want onion salad littl excel cook fresh water say thing start special way
Topic 3:
negreview great steak like wait tabl ami strip mon recommend chees watch server insid flavor la salmon reason filet scallop
Topic 4:
goodreview french fri right menu ask look better locat everyth appet baguett overal husband tender cut beauti room everyon yelp
Topic 5:
good seat view realli delici nice fountain got reserv best sit price eat sauc egg definit friend brunch wine didn


In [90]:
# table of 6 topics with 20 words from CODEWORD LDA
def get_topics(model, feature_names, no_top_words):
    topic_lists = []
    for topic_idx, topic in enumerate(model.components_):
        topic_list = []
        for i in topic.argsort()[:-no_top_words - 1:-1]:
            topic_list += [feature_names[i]]
        topic_lists.append(topic_list)
    return topic_lists
d = get_topics(lda_codeword, tf_feature_names, no_top_words)
import texttable as tt
tab = tt.Texttable()
headings = ['food','order','service','bad price','good environment','place']
tab.header(headings)
d0 = d[0]
d1 = d[1]
d2 = d[2]
d3 = d[3]
d4 = d[4]
d5 = d[5]
for row in zip(d0,d1,d2,d3,d4,d5):
    tab.add_row(row)

s = tab.draw()
print (s)

+------------+---------+-----------+-----------+------------------+----------+
|    food    |  order  |  service  | bad price | good environment |  place   |
+============+=========+===========+===========+==================+==========+
| place      | food    | servic    | negreview | goodreview       | good     |
+------------+---------+-----------+-----------+------------------+----------+
| vega       | order   | time      | great     | french           | seat     |
+------------+---------+-----------+-----------+------------------+----------+
| just       | restaur | patio     | steak     | fri              | view     |
+------------+---------+-----------+-----------+------------------+----------+
| bellagio   | come    | came      | like      | right            | realli   |
+------------+---------+-----------+-----------+------------------+----------+
| love       | dinner  | breakfast | wait      | menu             | delici   |
+------------+---------+-----------+-----------+----

In [91]:

# topic words = 20 topic = 10
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic %d:" % (topic_idx))
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 20
no_topics = 10

# # Run NMF
# nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

# # Run LDA
lda_codeword = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)
display_topics(lda_codeword, tf_feature_names, no_top_words)

Topic 0:
food just love tri mon amaz experi ask say baguett sandwich awesom said potato entre howev seafood qualiti super waffl
Topic 1:
steak order restaur come bread ami soup star pari minut chicken benedict perfectli attent cream beauti absolut hour larg cours
Topic 2:
servic patio fountain frite make want onion salad butter cook look water warm start toast trip mignon busi coffe mani
Topic 3:
negreview great like price chees insid better scallop escargot special sure know need waitress crispi light quit share anoth ate
Topic 4:
goodreview menu room face platter doesn simpl begin air wa crusti play alright mostli elsewher horribl mood caus 18 shade
Topic 5:
view realli nice outsid got best sit eat definit brunch wine didn day pretti dine crepe waiter wasn think outdoor
Topic 6:
bellagio tabl dinner strip peopl recommend server lunch flavor salmon locat favorit worth everi disappoint walk fantast portion hotel parti
Topic 7:
vega time wait sauc enjoy littl alway went night filet ever

In [93]:
# table of 10 topic with 20 words from CODEWORD LDA
def get_topics(model, feature_names, no_top_words):
    topic_lists = []
    for topic_idx, topic in enumerate(model.components_):
        topic_list = []
        for i in topic.argsort()[:-no_top_words - 1:-1]:
            topic_list += [feature_names[i]]
        topic_lists.append(topic_list)
    return topic_lists
d = get_topics(lda_codeword, tf_feature_names, no_top_words)
import texttable as tt
tab = tt.Texttable()
headings = ['food','order','service','bad price','good environment','place']
tab.header(headings)
d0 = d[0]
d1 = d[1]
d2 = d[2]
d3 = d[3]
d4 = d[4]
d5 = d[5]
for row in zip(d0,d1,d2,d3,d4,d5):
    tab.add_row(row)

s = tab.draw()
print (s)

+----------+-----------+----------+-----------+------------------+---------+
|   food   |   order   | service  | bad price | good environment |  place  |
+==========+===========+==========+===========+==================+=========+
| food     | steak     | servic   | negreview | goodreview       | view    |
+----------+-----------+----------+-----------+------------------+---------+
| just     | order     | patio    | great     | menu             | realli  |
+----------+-----------+----------+-----------+------------------+---------+
| love     | restaur   | fountain | like      | room             | nice    |
+----------+-----------+----------+-----------+------------------+---------+
| tri      | come      | frite    | price     | face             | outsid  |
+----------+-----------+----------+-----------+------------------+---------+
| mon      | bread     | make     | chees     | platter          | got     |
+----------+-----------+----------+-----------+------------------+---------+

In [95]:
# Compare with traditional LDA
# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_features=no_features, stop_words='english')
tf = tf_vectorizer.fit_transform(document_top1)
tf_feature_names = tf_vectorizer.get_feature_names()
no_topics = 6
no_top_words = 10
lda_codeword = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)
# topic words = 10
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic %d:" % (topic_idx))
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))


display_topics(lda_codeword, tf_feature_names, no_top_words)
def get_topics(model, feature_names, no_top_words):
    topic_lists = []
    for topic_idx, topic in enumerate(model.components_):
        topic_list = []
        for i in topic.argsort()[:-no_top_words - 1:-1]:
            topic_list += [feature_names[i]]
        topic_lists.append(topic_list)
    return topic_lists
d = get_topics(lda_codeword, tf_feature_names, no_top_words)
import texttable as tt
tab = tt.Texttable()
headings = ['food','order','service','price','environment','place']
tab.header(headings)
d0 = d[0]
d1 = d[1]
d2 = d[2]
d3 = d[3]
d4 = d[4]
d5 = d[5]
for row in zip(d0,d1,d2,d3,d4,d5):
    tab.add_row(row)

s = tab.draw()
print (s)

Topic 0:
salad medium rare burger bacon wow tuna old egg kale
Topic 1:
great food vegas bellagio place service patio view strip outside
Topic 2:
steak good ordered soup frites bread french onion cheese fries
Topic 3:
breakfast eggs crepe brunch benedict toast good coffee french ordered
Topic 4:
food service good table just time wait great place restaurant
Topic 5:
mon ami gabi good french great just wine vegas filet
+--------+----------+---------+-----------+-------------+--------+
|  food  |  order   | service |   price   | environment | place  |
+========+==========+=========+===========+=============+========+
| salad  | great    | steak   | breakfast | food        | mon    |
+--------+----------+---------+-----------+-------------+--------+
| medium | food     | good    | eggs      | service     | ami    |
+--------+----------+---------+-----------+-------------+--------+
| rare   | vegas    | ordered | crepe     | good        | gabi   |
+--------+----------+---------+-----------+-